# Variation

Plot the dispersion model for momentum power for various cases

In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
import crosslib
from ipywidgets import interact, IntSlider, FloatSlider

cache = {}

def damp(x):
    """ exponential damping function"""
    return np.exp(-x**2)

def damp_deriv(x):
    """ dD(x)/dx """
    return -2.0*x*np.exp(-x**2)

def load(isnp):
    if isnp in cache:
        return cache[isnp]
    
    d = {}
    
    # Load simulation data
    realspace = crosslib.load_power2d(isnp, 0) # lambda = 0 for real space
    zspace = crosslib.load_power2d(isnp, 1) # lambda = 1 for redshift space

    param = crosslib.load_param(isnp) # dict of paramters

    d['data/k'] = zspace['k']
    
    d['data/P_dd'] = realspace['summary']['Pdd']
    d['data/P_pd'] = realspace['summary']['Ppd']
    d['data/P_pp'] = realspace['summary']['Ppp']
    
    d['data/Ps_dd'] = zspace['summary']['Pdd']
    d['data/Ps_pd'] = zspace['summary']['Ppd']
    d['data/Ps_pp'] = zspace['summary']['Ppp']
    
    # Load for model
    bel = crosslib.load_theta_power_bel(isnp, Ptt_simple=False)
    
    idx = np.logical_and(0.01 <= bel['k'], bel['k'] <= 1.0)
    d['model/k'] = bel['k'][idx]
    
    # P_delta_delta, P_delta_theta, and P_theta_theta
    d['model/Pdd'] = bel['Pdd'][idx]
    d['model/Pdt'] = bel['Pdt'][idx]
    d['model/Ptt'] = bel['Ptt'][idx]
    d['model/f'] = param['f']
    d['model/s'] = 3.0 # damping paramter sigma_v 
    
    cache[isnp] = d
    
    return d
    
def compute_model(d, imu):
    model = {}
    
    mu = (imu + 0.5)/10
    k = d['model/k']
    f = d['model/f']
    s = d['model/s']
    Pdd = d['model/Pdd']
    Pdt = d['model/Pdt']
    Ptt = d['model/Ptt']


    D = damp(k*mu*s)
    Dp = damp_deriv(k*mu*s)

    # redshift-space Ps_dd
    Ps = (Pdd + 2*f*mu**2*Pdt + f**2*mu**4*Ptt)*D

    # realspace P_pd
    Pdp = (f*mu*Pdt + f**2*mu**3*Ptt)/k
    Pdps = Pdp*D + 0.5*s*Ps*Dp
    
    # pp
    Puu = f**2*mu**2*Ptt/k**2
    Prand = Pdd*s**2
    P1 = s*f*mu*Dp*(Pdt + f*mu**2*Ptt)/k
    P2 = 0.25*s**2*(Dp/D)**2*Ps
    
    model['k'] = k
    model['Ps_dd'] = Ps
    model['P_pd'] = (f*mu*Pdt)/k
    model['Ps_pd'] = Pdp*D + 0.5*s*Ps*Dp
    model['P_pp'] = Puu + Prand
    model['Ps_pp'] = (Puu + Prand)*D + P1 + P2
    
    return model


In [53]:

isnp = IntSlider(min=0, max=9, value=4)
imu = IntSlider(min=0, max=9, value=4)

@interact(isnp=isnp, imu=imu)
def plot(isnp, imu):
    isnp = '%03d' % isnp
    d = load(isnp)
    model = compute_model(d, imu)
    
    plt.figure(figsize=(12, 3))
    
    # dd
    plt.subplot(1, 3, 1)
    plt.title('$\\delta\\delta$')
    plt.xscale('log')
    plt.yscale('log')
    
    plt.plot(d['data/k'][:, imu], d['data/P_dd'][:, imu], 'x', markersize=2, color='black') 
    plt.plot(d['model/k'], d['model/Pdd'])
    
    plt.plot(d['data/k'][:, imu], d['data/Ps_dd'][:, imu], 'x', markersize=2, color='red') 
    plt.plot(model['k'], model['Ps_dd'], color='red', alpha=0.5)
    
    # pd
    plt.subplot(1, 3, 2)
    plt.title('$p\\delta$')
    plt.xscale('log')
    plt.yscale('log')
    
    plt.plot(d['data/k'][:, imu], np.abs(d['data/P_pd'][:, imu]), 'x',
             markersize=2, color='black')
    plt.plot(model['k'], np.abs(model['P_pd']))
    
    plt.plot(d['data/k'][:, imu], np.abs(d['data/Ps_pd'][:, imu]), 'x',
             markersize=2, color='red')
    plt.plot(model['k'], np.abs(model['Ps_pd']), color='red', alpha=0.5)

    # pp
    plt.subplot(1, 3, 3)
    plt.title('$pp$')
    plt.xscale('log')
    plt.yscale('log')
    
    plt.plot(d['data/k'][:, imu], np.abs(d['data/P_pp'][:, imu]), 'x', markersize=2,
            color='black')  
    plt.plot(model['k'], model['P_pp'])
    
    plt.plot(d['data/k'][:, imu], np.abs(d['data/Ps_pp'][:, imu]), 'x', markersize=2,
            color='red')  
    plt.plot(model['k'], model['Ps_pp'], color='red', alpha=0.5)

interactive(children=(IntSlider(value=4, description='isnp', max=9), IntSlider(value=4, description='imu', max…